In [1]:
import findspark
findspark.init()

In [7]:
!pip install pydeequ

  Using cached pandas-1.4.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.7 MB)
  Using cached numpy-1.22.2-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)


In [8]:
from pyspark.sql import SparkSession, Row

import pydeequ

spark = (SparkSession
    .builder
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())


Please set env variable SPARK_VERSION
22/02/15 23:43:33 WARN Utils: Your hostname, sarbajit-laptop resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
22/02/15 23:43:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/sarbajit/.ivy2/cache
The jars for the packages stored in: /home/sarbajit/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazon.deequ#deequ added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f2188b1e-1c8d-4311-8e3a-5caad4aeade9;1.0
	confs: [default]
	found com.amazon.deequ#deequ;1.2.2-spark-3.0 in central
	found org.scalanlp#breeze_2.12;0.13.2 in central
	found org.scalanlp#breeze-macros_2.12;0.13.2 in central
	found org.scala-lang#scala-reflect;2.12.1 in central
	found com.github.fommil.netlib#core;1.1.2 in central
	found net.sf.opencsv#opencsv;2.3 in centr

In [9]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Warning, "Review Check")

data = [
    ["Sarbajit","CSE", 19],
    ["Shalmoli","CSE", 20],
    ["Supriya","IT", 21],
    ["Shuvankar","ECE", 32],
]

df= spark.createDataFrame(data, schema=['name','dept','roll'])
df.show()

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(\
        check\
            .hasSize(lambda x: x >= 100)\
            .isUnique("roll")\
            .isUnique("name")\
            .isContainedIn("dept", ["CSE", "IT", "ECE"], lambda x: x!='CSE')\
        )\
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()
# checkResult_df.write.mode("overwrite").option('header',True).csv("../Data/PyDQ_Test/dfResult.csv")

+---------+----+----+
|     name|dept|roll|
+---------+----+----+
| Sarbajit| CSE|  19|
| Shalmoli| CSE|  20|
|  Supriya|  IT|  21|
|Shuvankar| ECE|  32|
+---------+----+----+

Python Callback server started!


TypeError: isContainedIn() takes 3 positional arguments but 4 were given

In [33]:
spark.stop()